<a href="https://colab.research.google.com/github/ich-julianopds/data_challenge/blob/main/data_challenge_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análise de Sentimentos no Twitter**
Esta é a segunda parte do projeto de Análise de Sentimentos no Twitter.
À princípio iria utilizar os dados armazenados na coleta do Mongo DB do processo anterior, porém, devido à imprevistos de tempo, acabei tendo que fazer o desafio nos dois últimos dias, por isso decidi utilizar um dataset do Kaggle já parcialmente tratado, sendo necessárias apenas modificações menores, com a limpeza e equalização dos dados, e como sabemos, é uma das partes que mais demanda tempo. Assim,  segue abaixo a sequência do projeto:

Como mencionado acima, os dados para uso foram do Kaggle, referentes à opinião de clientes em relação à uma companhia aérea americana, assim, o dataset está em inglês.
Me baseei também em um projeto do cientista de dados Odemir Depieri Jr.
Este projeto pode ser de ajuda por exemplo, para que as empresas possam fazer análise de opinião, satisfação, perfil de clientes, etc.
Utilizaremos como biblioteca principal o NLTK, que trata do processamento de Linguagem Natural.

In [7]:
pip install contractions

     |████████████████████████████████| 284 kB 5.3 MB/s 
     |████████████████████████████████| 321 kB 49.8 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85446 sha256=f9b1bb75083828ee7b57e8671e537cf6507baefbb1f65ff1777cb2e624b1b021
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [8]:
# Importação das bibliotecas necessárias

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import string
import re
import contractions

In [9]:
df_first = pd.read_csv('/content/Tweets.csv')

In [10]:
# Verificar as entradas

df_first.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [12]:
# 1460 linhas x 15 colunas
df_first.describe

<bound method NDFrame.describe of                  tweet_id  ...               user_timezone
0      570306133677760513  ...  Eastern Time (US & Canada)
1      570301130888122368  ...  Pacific Time (US & Canada)
2      570301083672813571  ...  Central Time (US & Canada)
3      570301031407624196  ...  Pacific Time (US & Canada)
4      570300817074462722  ...  Pacific Time (US & Canada)
...                   ...  ...                         ...
14635  569587686496825344  ...                         NaN
14636  569587371693355008  ...                         NaN
14637  569587242672398336  ...                         NaN
14638  569587188687634433  ...  Eastern Time (US & Canada)
14639  569587140490866689  ...                         NaN

[14640 rows x 15 columns]>

Antes de começar a tratar os valores nulos e demais, podemos já excluir as colunas que não serão necessárias para nosso modelo:

In [13]:
df = df_first.drop(['tweet_id', 'retweet_count', 'tweet_coord', 'tweet_created', 'tweet_location', 'name', 'user_timezone'], axis = 1)

In [14]:
df.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,negativereason_gold,text
0,neutral,1.0000,NaN,NaN,Virgin America,NaN,NaN,@VirginAmerica What @dhepburn said.
1,positive,0.3486,NaN,0.0000,Virgin America,NaN,NaN,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,NaN,NaN,Virgin America,NaN,NaN,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,NaN,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,NaN,@VirginAmerica and it's a really big bad thing...


Ao analisar o dataset, temos três sentimentos que serão analisados: positivos, negativos e neutros. Sendo que aqueles que deram notas negativas, têm a justificativa na coluna "negativereason".

In [16]:
df['negativereason'] = df['negativereason'].fillna('')
df['negativereason_confidence'] = df['negativereason_confidence'].fillna('')
df['airline_sentiment_gold'] = df['airline_sentiment_gold'].fillna('')
df['negativereason_gold'] = df['negativereason_gold'].fillna('')

In [17]:
df.shape

(14640, 8)

In [18]:
df.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,negativereason_gold,text
0,neutral,1.0000,,,Virgin America,,,@VirginAmerica What @dhepburn said.
1,positive,0.3486,,0,Virgin America,,,@VirginAmerica plus you've added commercials t...
2,neutral,0.6837,,,Virgin America,,,@VirginAmerica I didn't today... Must mean I n...
3,negative,1.0000,Bad Flight,0.7033,Virgin America,,,@VirginAmerica it's really aggressive to blast...
4,negative,1.0000,Can't Tell,1,Virgin America,,,@VirginAmerica and it's a really big bad thing...


In [21]:
def Clean_Text(Texto):

  if Texto:
    Texto = contractions.fix(Texto)

    Texto = ' '.join(Texto.split('.'))

    Texto = re.sub(r'\s+', ' ',
                   re.sub('[^A-Za-z0-9]', ' ',
                          Texto.strip().lower())).strip()

    Texto = re.sub(r'\W+', ' ', Texto.strip().lower()).strip()

    Texto = [Palavra for Palavra in Texto.split()]

    return Texto


df['text'] = df['text'].apply(lambda Linha : ' '.join(Clean_Text(Linha)))

In [26]:
nltk.download(['names', 'stopwords', 'vader_lexicon'])

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [28]:
from nltk.sentiment import SentimentIntensityAnalyzer

Função_SIA = SentimentIntensityAnalyzer()

print('I hate chocolate: ', Função_SIA.polarity_scores('I hate chocolate'), '\n')
print('I love chocolate: ', Função_SIA.polarity_scores('I love chocolate'), '\n')

I hate chocolate:  {'neg': 0.787, 'neu': 0.213, 'pos': 0.0, 'compound': -0.5719} 

I love chocolate:  {'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'compound': 0.6369} 



In [36]:
Textos = df['text'].tolist()

In [40]:
Scores_Negativos = []
Scores_Neutros = []
Scores_Positivos = []
Combinacao_Scores = []
Final = []


for Frases in Textos:
  Score_Emocao = Função_SIA.polarity_scores(Frases)

  Scores_Negativos.append(Score_Emocao['neg'])
  Scores_Positivos.append(Score_Emocao['pos'])
  Scores_Neutros.append(Score_Emocao['neu'])
  Combinacao_Scores.append(Score_Emocao['compound'])
  if Score_Emocao['compound'] > 0:
    Final.append('positive')

  elif Score_Emocao['compound'] < 0:
    Final.append('negative')
  else:
    Final.append('neutral')

In [41]:
df['negative_score'] = Scores_Negativos
df['positive_score'] = Scores_Positivos
df['neutral_score'] = Scores_Neutros
df['compound_score'] = Combinacao_Scores
df['Final'] = Final

In [44]:
# Avaliação de como ficou o processo:

df[['text','negative_score', 'positive_score', 'neutral_score', 'compound_score', 'Final']].head(20)

,text,negative_score,positive_score,neutral_score,compound_score,Final
0,virginamerica what dhepburn said,0.000,0.000,1.000,0.0000,neutral
1,virginamerica plus you have added commercials ...,0.000,0.000,1.000,0.0000,neutral
2,virginamerica i did not today must mean i need...,0.000,0.000,1.000,0.0000,neutral
3,virginamerica it is really aggressive to blast...,0.216,0.123,0.661,-0.2716,negative
4,virginamerica and it is a really big bad thing...,0.296,0.000,0.704,-0.5829,negative
5,virginamerica seriously would pay 30 a flight ...,0.238,0.069,0.693,-0.5945,negative
6,virginamerica yes nearly every time i fly vx t...,0.000,0.172,0.828,0.4019,positive
7,virginamerica really missed a prime opportunit...,0.142,0.175,0.683,0.1458,positive
8,virginamerica well i did not but now i do d,0.000,0.205,0.795,0.1406,positive
9,virginamerica it was amazing and arrived an ho...,0.000,0.340,0.660,0.7717,positive


In [45]:
pip install transformers

     |████████████████████████████████| 2.8 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 39.3 MB/s 
     |████████████████████████████████| 636 kB 68.7 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 3.3 MB 35.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [57]:
# Iportamos a biblioteca transformers para fazer a mineração da emoção e avaliremos sua acurácia

from transformers import pipeline

Funcao_Classificacao = pipeline('sentiment-analysis')

Funcao_Classificacao('i hate chocolate')[0]['label']

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


'NEGATIVE'

In [58]:
Funcao_Classificacao('i love chocolate')[0]['label']

'POSITIVE'

In [61]:
Funcao_Classificacao('i dont like chocolate')[0]['label']

'NEGATIVE'

In [62]:
Funcao_Classificacao('i really love airplanes')[0]['label']

'POSITIVE'

Após a tratativa e aplicação dos modelos das próprias bibliotecas, foi possível fazer a análise de sentimentos de posts do Twitter.
Esta análise foi feita com os modelos das prórpias libs importadas. Poderíamos fazer também com qualquer dataset que coletarmos no Twitter, após o processo de ETL do arquivos que armazenamos em um banco de dados, filtrarmos os dados, transformarmos os arquivos json, assim teríamos a base inicial para trabalhar melhor. Após a transformação em df, fica mais fácil a utilização com o Pandas.
Há também a possibilidade de fazer a implementação de análise de sentimentos em modelos em tempo real com o consumo da API, porém, exigiriam mais recursos, sendo necessária uma lib como o PySpark por exemplo.